In [1]:
import os
import time

import duckdb
import httpx
import pandas as pd
import matplotlib.pyplot as plt
from retry import retry
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
import json


def write_json(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [ ]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_transaction(network, collection_id, continuation="", type=["sale", "transfer", "mint"], last_timestamp: int = -1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/collections/activity/v6"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "collection": collection_id,
        "limit": 50,
        "types": type,
    }

    if continuation != "":
        params["continuation"] = continuation

    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()
    
    if last_timestamp != -1:
        resp["activities"] = [activity for activity in resp["activities"] if activity["timestamp"] > last_timestamp]
    
    if len(resp["activities"]) == 0:
        resp["continuation"] = None
    
    for activity in resp["activities"]:
        activity["network"] = network
        activity["collection_id"] = collection_id
        
    return resp

In [ ]:
resp = get_transaction("ethereum", "0x06012c8cf97bead5deae237070f9587f8e7a266d", type=["mint"])
write_json(resp, "output/mint3.json")

In [ ]:
resp = get_transaction("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", type=["mint"], last_timestamp=1668322751)
resp

In [11]:
df = duckdb.read_json("output/mint/*.json").to_df()
df = pd.concat([pd.json_normalize(row["activities"]) for idx, row in df.iterrows()]).reset_index(drop=True)
df

/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_89708/2450108895.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([pd.json_normalize(row["activities"]) for idx, row in df.iterrows()]).reset_index(drop=True)


,type,fromAddress,toAddress,amount,timestamp,createdAt,contract,txHash,logIndex,batchIndex,...,token.isNsfw,token.tokenName,token.tokenImage,token.rarityScore,token.rarityRank,collection.collectionId,collection.isSpam,collection.isNsfw,collection.collectionName,collection.collectionImage
0,mint,0x0000000000000000000000000000000000000000,0x7fb6123f947dff6bd83f2f7a05fa35367abb7ced,1,1718764883,2024-06-19T02:41:26.488Z,0x524cab2ec69124574082676e6f654a18df49a048,0x28939514c1fdabcac94c3023b6a4c2448ede1d9e035a...,14,1,...,False,Lil Pudgy #7451,https://img.reservoir.tools/images/v2/mainnet/...,159.085,15994.0,0x524cab2ec69124574082676e6f654a18df49a048,False,False,Lil Pudgys,https://img.reservoir.tools/images/v2/mainnet/...
1,mint,0x0000000000000000000000000000000000000000,0x9a430003baed68f7cf460de24182ee8f83faff00,1,1718847635,2024-06-20T01:40:37.301Z,0x8821bee2ba0df28761afff119d66390d594cd280,0xd30331d96a452ef8e253092322fbc8dbef3fac2179b4...,254,1,...,False,DeGod #9939,https://img.reservoir.tools/images/v2/mainnet/...,NaN,NaN,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
2,mint,0x0000000000000000000000000000000000000000,0x5bbda203d20d8d479a3bf4543f4d64e9f0ee2cec,1,1718838779,2024-06-19T23:13:01.802Z,0x8821bee2ba0df28761afff119d66390d594cd280,0xdcbb23d2f75d783ffb60a29aecb2103d1b4301fd50de...,347,1,...,False,DeGod #1379,https://img.reservoir.tools/images/v2/mainnet/...,144.818,1584.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
3,mint,0x0000000000000000000000000000000000000000,0x5b092fd14c760fc2b4ac57af07fd6fcc7f7d9080,1,1718836571,2024-06-19T22:36:15.153Z,0x8821bee2ba0df28761afff119d66390d594cd280,0x884f2788f7a31cd1e88d6417caee9f38e3a18a5bc054...,270,1,...,False,DeGod #7096,https://img.reservoir.tools/images/v2/mainnet/...,216.209,210.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
4,mint,0x0000000000000000000000000000000000000000,0xc665617d340108e35ecfcbad37d448b47b1c54d3,1,1718834303,2024-06-19T21:58:26.391Z,0x8821bee2ba0df28761afff119d66390d594cd280,0x706e794d7d4f9bcd752cd6ce959d788cf4d40ee2ee45...,115,1,...,False,DeGod #1248,https://img.reservoir.tools/images/v2/mainnet/...,113.456,3094.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
5,mint,0x0000000000000000000000000000000000000000,0xc665617d340108e35ecfcbad37d448b47b1c54d3,1,1718834291,2024-06-19T21:58:13.403Z,0x8821bee2ba0df28761afff119d66390d594cd280,0xdac60abc3c9b141245c891137a4ab0476f48a6c8623b...,145,1,...,False,DeGod #4877,https://img.reservoir.tools/images/v2/mainnet/...,71.633,5498.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
6,mint,0x0000000000000000000000000000000000000000,0x51f32f53f6ff13de875c8f3ca9cc88b11bf4ad17,1,1718828147,2024-06-19T20:15:53.637Z,0x8821bee2ba0df28761afff119d66390d594cd280,0xd6b5db9b3111fe9829e69bbdf7f2b4eb4d6f37054f43...,1045,1,...,False,DeGod #8216,https://img.reservoir.tools/images/v2/mainnet/...,133.774,1793.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
7,mint,0x0000000000000000000000000000000000000000,0x62283dee96a3dfec6c2148652eaec6723cb98248,1,1718824967,2024-06-19T19:22:49.553Z,0x8821bee2ba0df28761afff119d66390d594cd280,0xa1e1b176c1aa7692a8ac68731ccdb6cb27c11878e34b...,302,1,...,False,DeGod #1427,https://img.reservoir.tools/images/v2/mainnet/...,102.663,3918.0,0x8821bee2ba0df28761afff119d66390d594cd280,False,False,DeGods,https://img.reservoir.tools/images/v2/mainnet/...
8,mint,0x0000000000000000000000000000000000000000,0x519d4add0a85af09a92cde515419e8c6a215275d,1,1718799563,2024-06-19T12:19:25.572Z,0x8821bee2ba0df28761afff119d66390d594cd280,0x2d340a4bd010887ff799e9cde081eb286a10a93ad282...,285,1,...,False,DeGod #1553,https://img.reservoir.tools

In [14]:
df.columns

Index(['type', 'fromAddress', 'toAddress', 'amount', 'timestamp', 'createdAt',
       'contract', 'txHash', 'logIndex', 'batchIndex', 'isAirdrop', 'network',
       'collection_id', 'price.currency.contract', 'price.currency.name',
       'price.currency.symbol', 'price.currency.decimals', 'price.amount.raw',
       'price.amount.decimal', 'price.amount.usd', 'price.amount.native',
       'token.tokenId', 'token.isSpam', 'token.isNsfw', 'token.tokenName',
       'token.tokenImage', 'token.rarityScore', 'token.rarityRank',
       'collection.collectionId', 'collection.isSpam', 'collection.isNsfw',
       'collection.collectionName', 'collection.collectionImage'],
      dtype='object')

In [17]:
df.loc[df['timestamp'] > 1717745231]

df[["network", "collection_id", "token.tokenId", "token.tokenName",  "timestamp"]]

,network,collection_id,timestamp
0,ethereum,0x524cab2ec69124574082676e6f654a18df49a048,1718764883
1,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718847635
2,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718838779
3,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718836571
4,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718834303
5,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718834291
6,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718828147
7,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718824967
8,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718799563
9,ethereum,0x8821bee2ba0df28761afff119d66390d594cd280,1718786399


### Transaction Sync

In [ ]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_transfer_bulk(network, collection_id, continuation="", start_timestamp=-1, end_timestamp=-1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/transfers/bulk/v2"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "contract": collection_id,
        "limit": 1000,
    }

    if continuation != "":
        params["continuation"] = continuation
        
    if start_timestamp != -1:
        params["startTimestamp"] = start_timestamp
    
    if end_timestamp != -1:
        params["endTimestamp"] = end_timestamp

    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()
    for transfer in resp["transfers"]:
        transfer["network"] = network
        transfer["collection_id"] = collection_id
    

    return resp

In [ ]:
resp = get_transfer_bulk("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", start_timestamp=1718693423)
write_json(resp, "output/transfer_bulk.json")

In [ ]:
start_timestamp = 1718693423
end_timestamp = int(time.time())
continuation = ""

while True:
    resp = get_transfer_bulk("ethereum", "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d", start_timestamp=start_timestamp, end_timestamp=end_timestamp, continuation=continuation)
    write_json(resp, f"output/transfer_bulk_{start_timestamp}_{continuation}.json")
    
    continuation = resp["continuation"]
    
    if continuation in [None, ""]:
        break

In [ ]:
df = duckdb.read_json("output/*.json")

In [ ]:
transfers = df["transfers"].fetchdf()
transfers

In [ ]:
flatten = pd.concat([pd.json_normalize(row["transfers"]) for idx, row in transfers.iterrows()])

In [ ]:
flatten

In [ ]:
flatten.shape

In [ ]:
# Assuming the timestamps are stored in the 'timestamp' column of the 'flatten' DataFrame
timestamps = flatten['timestamp']

# Plotting the histogram of timestamps
plt.hist(timestamps)
plt.xlabel('Timestamp')
plt.ylabel('Frequency')
plt.title('Distribution of Timestamps')
plt.show()

In [ ]:
flatten.duplicated().sum()

In [ ]:
1718693423

In [ ]:
time.time()

In [ ]:
time.time() - 1718693423

In [ ]:
(time.time() - 1718693423) / 60 / 60 / 24

In [ ]:
flatten.loc[flatten["timestamp"] < 1718769000]

In [ ]:
flatten.iloc[0].txHash

In [ ]:
flatten["token.tokenId"].value_counts()

### Sale Sync

In [7]:
@retry(Exception, tries=3, delay=1, backoff=2, jitter=3)
def get_sale(network, collection_id, continuation="", start_timestamp=-1, end_timestamp=-1):
    networks = {"ethereum": "api", "polygon": "api-polygon"}

    url = f"https://{networks[network]}.reservoir.tools/sales/v6"

    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    }
    params = {
        "collection": collection_id,
        "limit": 1000
    }

    if continuation != "":
        params["continuation"] = continuation

    if start_timestamp != -1:
        params["startTimestamp"] = start_timestamp
    
    if end_timestamp != -1:
        params["endTimestamp"] = end_timestamp
    
    resp = httpx.get(url, params=params, headers=headers, timeout=30)

    # 200번 이외 경우에 에러를 반환하여 재시도를 하도록 함.
    resp.raise_for_status()

    resp = resp.json()

    for sale in resp["sales"]:
        sale["network"] = network
        sale["collection_id"] = collection_id
    
    return resp

In [9]:
network = "ethereum"
collection_id = "0xed5af388653567af2f388e6224dc7c4b3241c544"
continuation = ""
start_timestamp = 1718693423
end_timestamp = int(time.time())

timestamp = int(time.time())

# 14분 30초 이후
while int(time.time()) - timestamp < 870:
    sales = get_sale(network, collection_id, continuation, start_timestamp=start_timestamp, end_timestamp=end_timestamp)
    write_json(sales, f"output/sale/sale_{continuation}.json")
    
    continuation = sales["continuation"]
    
    if sales["continuation"] is None:
        break